# Snorkel

## Introduction

The goal of this lab is to introduce students to the [Snorkel](http://www.snorkel.org) tool and the possibilities of programmatic label generation using the weak-supervised learning paradigm.

In order to use weakly supervised learning to generate labels, it is necessary to create three datasets:

- **train set**: which does not have any labels
- **validation set**: used for hyperparameter optimization, has labels
- **test set**: used only for final model evaluation, has labels

## Labeling functions

The first step will be to load the dataset and split it into a train set and a test set. Since in our set all SMS have a label, we will simulate a weakly supervised learning problem by randomly removing 80% of the labels. Additionally, Snorkel requires numeric labels, so we need to recode the values.

In [16]:
!head datasets/smsspamcollection.csv

ham	Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
ham	Ok lar... Joking wif u oni...
spam	Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham	U dun say so early hor... U c already then say...
ham	Nah I don't think he goes to usf, he lives around here though
spam	FreeMsg Hey there darling it's been 3 week's now and no word back! I'd like some fun you up for it still? Tb ok! XxX std chgs to send, £1.50 to rcv
ham	Even my brother is not like to speak with me. They treat me like aids patent.
ham	As per your request 'Melle Melle (Oru Minnaminunginte Nurungu Vettam)' has been set as your callertune for all Callers. Press *9 to copy your friends Callertune
spam	WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.
spam	H

In [17]:
import pandas as pd
import numpy as np

pd.set_option('max_colwidth', 600)

SPAM = 1
HAM = 0
ABSTAIN = -1

df = pd.read_csv('./datasets/smsspamcollection.csv', sep='\t', header=None, names=['old_label', 'text'])

df['label'] = df.old_label.apply(lambda x: SPAM if x == 'spam' else HAM)

df.loc[df.sample(frac=0.8).index, 'label'] = ABSTAIN
df.drop(columns=['old_label'], inplace=True)

df.head()

,text,label
0,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's,1
3,U dun say so early hor... U c already then say...,-1
4,"Nah I don't think he goes to usf, he lives around here though",-1


In [18]:
abstain_idx = df.label == ABSTAIN

df_train = df[abstain_idx]
df_test = df[~abstain_idx]

### Simple keyword search

As a first example, we will use a search for the words "check" and "free" in SMS content

In [19]:
from snorkel.labeling import labeling_function

@labeling_function()
def check(sms):
    return SPAM if "check" in sms.text.lower() else ABSTAIN

@labeling_function()
def free(sms):
    return SPAM if "free" in sms.text.lower() else ABSTAIN

The next step is to apply the labeling functions to the train set.

In [20]:
from snorkel.labeling import PandasLFApplier

lfs = [check, free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 85305.93it/s]


The result of applying the set of labeling functions to the train set is a matrix of size $m \times n$, where $m$ is the number of examples and $n$ is the number of labeling functions. The matrix contains the result of applying each function to each example.

In [21]:
L_train

array([[-1, -1],
       [-1, -1],
       [-1,  1],
       ...,
       [-1, -1],
       [-1,  1],
       [-1, -1]])

In [22]:
df_train.iloc[1,:]

text     Nah I don't think he goes to usf, he lives around here though
label                                                               -1
Name: 4, dtype: object

The simplest way to analyze this is to determine the coverage of labeling functions (i.e., the percentage of cases for which the function returned a result other than `ABSTAIN'.

In [23]:
coverage_check, coverage_free = (L_train != ABSTAIN).mean(axis=0)

print(f"Coverage for check(): {coverage_check * 100:.1f}%")
print(f"Coverage for free(): {coverage_free * 100:.1f}%")

Coverage for check(): 1.0%
Coverage for free(): 4.4%


Fortunately, Snorkel offers additional tools that allow for deeper analysis of the result of labeling functions.

In [24]:
from snorkel.labeling import LFAnalysis

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.0,0.0
free,1,[1],0.043517,0.0,0.0


The meaning of each column is as follows:
- `Polarity`: the set of labels returned by the function
- `Coverage`: the percentage of examples for which the function returns a value other than `ABSTAIN`
- Overlaps: the percentage of examples for which at least one other labeling function returned a value
- Conflicts: the percentage of examples for which at least one other labeling function returned a different value

If the train set contained labels, the method would also return:
- `Correct`: the number of correct labels
- `Incorrect`: number of incorrect labels
- `Empirical Accuracy`: the percentage of correct labels

Let's check the examples labeled by the `free()` function as spam

In [25]:
df_train.iloc[L_train[:,1] == SPAM].sample(frac=0.1)

,text,label
3972,Free video camera phones with Half Price line rental for 12 mths and 500 cross ntwk mins 100 txts. Call MobileUpd8 08001950382 or Call2OptOut/674&,-1
4587,"Mila, age23, blonde, new in UK. I look sex with UK guys. if u like fun with me. Text MTALK to 69866.18 . 30pp/txt 1st 5free. £1.50 increments. Help08718728876",-1
3423,Freemsg: 1-month unlimited free calls! Activate SmartCall Txt: CALL to No: 68866. Subscriptn3gbp/wk unlimited calls Help: 08448714184 Stop?txt stop landlineonly,-1
1507,Thanks for the Vote. Now sing along with the stars with Karaoke on your mobile. For a FREE link just reply with SING now.,-1
270,Ringtone Club: Get the UK singles chart on your mobile each week and choose any top quality ringtone! This message is free of charge.,-1
2189,FREE camera phones with linerental from 4.49/month with 750 cross ntwk mins. 1/2 price txt bundle deals also avble. Call 08001950382 or call2optout/J MF,-1
1229,FREE entry into our £250 weekly comp just send the word ENTER to 84128 NOW. 18 T&C www.textcomp.com cust care 08712405020.,-1
1777,Call FREEPHONE 0800 542 0578 now!,-1
5342,"u r subscribed 2 TEXTCOMP 250 wkly comp. 1st wk?s free question follows, subsequent wks charged@150p/msg.2 unsubscribe txt STOP 2 84128,custcare 08712405020",-1
2980,"Xmas Offer! Latest Motorola, SonyEricsson & Nokia & FREE Bluetooth or DVD! Double Mins & 1000 Txt on Orange. Call MobileUpd8 on 08000839402 or call2optout/4QF2",-1


It seems that the phrase "call now" is also a good indicator for spam. So let's add one more labeling function.

In [26]:
@labeling_function()
def call_now(sms):
    return SPAM if "call now" in sms.text.lower() else ABSTAIN

lfs = [check, free, call_now]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

100%|██████████| 4458/4458 [00:00<00:00, 64499.49it/s]


In [27]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.000000,0.0
free,1,[1],0.043517,0.001346,0.0
call_now,2,[1],0.004262,0.001346,0.0


Let's see which examples were labeled as spam by the `call_now()` function but omitted by `free()`.

In [28]:
from snorkel.analysis import get_label_buckets

buckets = get_label_buckets(L_train[:, 1], L_train[:, 2]) #we apply only second and third labeling function
buckets

{(-1, -1): array([   0,    1,    3, ..., 4454, 4455, 4457]),
 (1,
  -1): array([   2,    6,    9,   47,   61,   76,  111,  118,  145,  146,  153,
         182,  213,  214,  236,  254,  290,  304,  318,  362,  392,  397,
         419,  490,  497,  516,  537,  551,  579,  636,  640,  654,  655,
         707,  734,  772,  816,  821,  830,  891,  971, 1000, 1002, 1037,
        1099, 1109, 1121, 1176, 1187, 1214, 1224, 1225, 1234, 1249, 1255,
        1279, 1308, 1323, 1331, 1352, 1380, 1431, 1439, 1442, 1444, 1495,
        1536, 1543, 1617, 1673, 1676, 1696, 1706, 1734, 1758, 1794, 1851,
        1861, 1883, 1900, 1953, 1986, 2015, 2066, 2101, 2117, 2132, 2243,
        2278, 2286, 2300, 2302, 2318, 2362, 2386, 2395, 2405, 2415, 2494,
        2499, 2502, 2515, 2543, 2572, 2610, 2645, 2685, 2704, 2721, 2732,
        2733, 2749, 2788, 2798, 2816, 2922, 2953, 2967, 2982, 3007, 3017,
        3051, 3078, 3083, 3110, 3134, 3158, 3161, 3176, 3182, 3206, 3209,
        3236, 3251, 3262, 3269, 3274, 33

In [29]:
df_train.iloc[buckets[(ABSTAIN, SPAM)]]

,text,label
876,"Shop till u Drop, IS IT YOU, either 10K, 5K, £500 Cash or £100 Travel voucher, Call now, 09064011000. NTT PO Box CR01327BT fixedline Cost 150ppm mobile vary",-1
1502,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
2170,"Shop till u Drop, IS IT YOU, either 10K, 5K, £500 Cash or £100 Travel voucher, Call now, 09064011000. NTT PO Box CR01327BT fixedline Cost 150ppm mobile vary",-1
2850,"YOUR CHANCE TO BE ON A REALITY FANTASY SHOW call now = 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national = rate call",-1
2992,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
3167,"HOT LIVE FANTASIES call now 08707509020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870..k",-1
3893,URGENT This is our 2nd attempt to contact U. Your £900 prize from YESTERDAY is still awaiting collection. To claim CALL NOW 09061702893. ACL03530150PM,-1
4073,Loans for any purpose even if you have Bad Credit! Tenants Welcome. Call NoWorriesLoans.com on 08717111821,-1
4108,"HOT LIVE FANTASIES call now 08707500020 Just 20p per min NTT Ltd, PO Box 1327 Croydon CR9 5WB 0870 is a national rate call",-1
4283,U can call now...,-1


In [30]:
LFAnalysis(L=L_train, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.000000,0.0
free,1,[1],0.043517,0.001346,0.0
call_now,2,[1],0.004262,0.001346,0.0


#### assignment

Write a labeling function that marks as spam all messages containing the word "HOT" written in capitals.

In [31]:
@labeling_function()
def hot(sms):
    return SPAM if "HOT" in sms.text else ABSTAIN

### Searching based on a regular expression

Another type of labeling function is one that uses regexp to find specific expressions.

In [32]:
import re

@labeling_function()
def regex_I_am_free(sms):
    if re.search(r"I\s.*free", sms.text, flags=re.I): #flags = re.I -> ignore case
        return HAM
    elif re.search(r"free", sms.text, flags=re.I):
        return SPAM
    else:
        return ABSTAIN

lfs = [check, free, call_now, regex_I_am_free]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 29859.51it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.000000,0.000000
free,1,[1],0.043517,0.043517,0.006057
call_now,2,[1],0.004262,0.001346,0.000000
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.006057


Let's compare examples that the `free()` function labels as spam and the `regex_I_am_free()` function considers valid.

In [33]:
buckets = get_label_buckets(L_train[:, 1], L_train[:, 3])
df_train.iloc[buckets[(SPAM, HAM)]].sample(10, random_state=1)

,text,label
3855,oh ya... Got hip hop open. Haha i was thinking can go for jazz then zoom to cine... Actually tonight i'm free leh... And there's a kb lesson tonight,-1
3526,I not free today i haf 2 pick my parents up tonite...,-1
3951,I got to video tape pple type in message lor. U so free wan 2 help me? Hee... Cos i noe u wan 2 watch infernal affairs so ask u along. Asking shuhui oso.,-1
1364,"Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping. Can you just please find some other way to get what you wanted us to get. Please forgive me. You can reply free via yahoo messenger.",-1
4949,"Hi this is Amy, we will be sending you a free phone number in a couple of days, which will give you an access to all the adult parties...",-1
3216,I want snow. It's just freezing and windy.,-1
4386,Do you want a New Nokia 3510i Colour Phone Delivered Tomorrow? With 200 FREE minutes to any mobile + 100 FREE text + FREE camcorder Reply or Call 8000930705,-1
4471,"Lemme know when I can swing by and pick up, I'm free basically any time after 1 all this semester",-1
1663,Hi if ur lookin 4 saucy daytime fun wiv busty married woman Am free all next week Chat now 2 sort time 09099726429 JANINExx Calls£1/minMobsmoreLKPOBOX177HP51FL,-1
948,Hey i booked the kb on sat already... what other lessons are we going for ah? Keep your sat night free we need to meet and confirm our lodging,-1


#### assignment

Write a labeling function that will mark as spam all messages containing any amounts specified with a currency symbol ($99, £1.50)

In [34]:
@labeling_function()
def contains_money(sms):
    return SPAM if re.search("[$£](([1-9][0-9]*)|(0))([.,][0-9]+)?", sms.text) else ABSTAIN

### Searching based on heuristics

A simple heuristic to find spam is to assume that if more than 10% of the message text is written in capitals, there is a good chance it is spam.

In [35]:
@labeling_function()
def has_many_uppercase_words(sms):
    percentage_uppercase = sum([word.isupper() for word in sms.text.split()]) / len(sms.text.split())
    
    return SPAM if percentage_uppercase > 0.1 else ABSTAIN

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 28528.46it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.002243,0.000000
free,1,[1],0.043517,0.043517,0.006057
call_now,2,[1],0.004262,0.003813,0.000000
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.006057
has_many_uppercase_words,4,[1],0.184388,0.020637,0.001346


#### assignment

Write a labeling function that marks as valid those messages that are shorter than 10 words and do not contain any word written in capitals.

In [111]:
@labeling_function()
def short_and_no_uppercase(sms):
    toks = sms.text.split()
    if len(toks) >= 10: return ABSTAIN
    for tok in toks:
        if tok == tok.upper(): return ABSTAIN
    return HAM

lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, short_and_no_uppercase]
applier = PandasLFApplier(lfs = lfs)
L_train = applier.apply(df = df_train)

LFAnalysis(L = L_train, lfs = lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 24727.81it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.003365,0.001122
free,1,[1],0.043517,0.043517,0.008075
call_now,2,[1],0.004262,0.004038,0.000449
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.008075
has_many_uppercase_words,4,[1],0.184388,0.020637,0.001346
short_and_no_uppercase,5,[0],0.277254,0.003365,0.003365


### Using an external statistical model

When labeling data, you can use external models whose response can be important information for deciding how to label an example. Snorkel has several built-in integrations in the form of the `Preprocessor` interface, in the example below we will use the `SpaCy` library to perform additional grammatical analysis of the text. However, you will need to download the English language model.

In [37]:
#!python3 -m spacy download en_core_web_sm

In [38]:
!python3 -m spacy validate

✔ Loaded compatibility table

================= Installed pipeline packages (spaCy v3.7.5) =================
ℹ spaCy installation:
/Users/Kuba/Library/Python/3.9/lib/python/site-packages/spacy

NAME             SPACY            VERSION                            
en_core_web_sm   >=3.7.2,<3.8.0   3.7.1   ✔
en_core_web_md   >=3.7.2,<3.8.0   3.7.1   ✔



In [39]:
import spacy

nlp = spacy.load('en_core_web_sm')

In [40]:
_text = """I don't England is a country that is part of the United Kingdom. 
It shares land borders with Wales to its west and Scotland to its north. 
The Irish Sea lies northwest of England and the Celtic Sea to the southwest. 
England is separated from continental Europe by the North Sea to the east and the 
English Channel to the south. The country covers five-eighths of the island of 
Great Britain, which lies in the North Atlantic, and includes over 100 smaller islands, 
such as the Isles of Scilly and the Isle of Wight."""

doc = nlp(_text)

for e in doc.ents: #ents == named entities
    print(e.text, e.label_)

England GPE
the United Kingdom GPE
Wales ORG
Scotland GPE
The Irish Sea LOC
England GPE
the Celtic Sea LOC
England GPE
Europe LOC
the North Sea LOC
English LANGUAGE
five-eighths CARDINAL
Great Britain GPE
the North Atlantic LOC
over 100 CARDINAL
the Isles of Scilly GPE
the Isle of Wight GPE


In [41]:
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy_ = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

Assume that short text messages in which a reference to a specific person appears are not spam.

In [42]:
df_train.columns

Index(['text', 'label'], dtype='object')

In [55]:
@labeling_function(pre=[spacy_])
def has_person(sms):
    if len(sms.doc) < 20 and any([ent.label_ == "PERSON" for ent in sms.doc.ents]):
        return HAM
    else:
        return ABSTAIN

In [57]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:00<00:00, 23203.89it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.003365,0.001122
free,1,[1],0.043517,0.043517,0.006954
call_now,2,[1],0.004262,0.003813,0.000000
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.006954
has_many_uppercase_words,4,[1],0.184388,0.026469,0.007178
has_person,5,[0],0.050022,0.007851,0.007851


Another example of pre-processing data for labeling would be determining the average word frequency of a document. Below we define a function that determines the average word frequency and we decorate it as an example of a pre-processor. When a text message is sent to the next labeling function, the pre-processor will populate the text message with the average word frequency and, based on that, the labeling function will make a decision (we assume that if the text message contains many rare words then it is spam).

In [58]:
from wordfreq import zipf_frequency
from snorkel.preprocess import preprocessor

@preprocessor(memoize=True)
def avg_word_freq(sms):
    sms.avg_word_freq = sum([zipf_frequency(word, 'en') for word in sms.text.split()]) / len(sms.text.split())
    
    return sms

In [59]:
@labeling_function(pre=[avg_word_freq])
def many_rare_words(sms):
    return ABSTAIN if sms.avg_word_freq >= 4 else SPAM

In [60]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, many_rare_words, has_person]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:01<00:00, 4430.10it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.003813,0.001122
free,1,[1],0.043517,0.043517,0.006954
call_now,2,[1],0.004262,0.003813,0.000000
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.006954
has_many_uppercase_words,4,[1],0.184388,0.044639,0.007178
many_rare_words,5,[1],0.062135,0.033423,0.008973
has_person,6,[0],0.050022,0.015029,0.015029


In [61]:
df_train.iloc[L_train[:,6] == SPAM].sample(frac=0.1)

,text,label


#### assignment

Write a labeling function that marks messages containing more than 3 adjectives as spam. Use the SpaCy library for pre-processing. 

__Hint__: the following example shows how to read the part-of-speech label for each token from the message being analyzed. For information on all token properties recognized by SpaCy, see [API documentation](https://spacy.io/api/token)

In [62]:
import spacy

nlp = spacy.load('en_core_web_sm')

sms = "Yetunde, i'm sorry but moji and i seem too busy to be able to go shopping."

for token in nlp(sms):
    print(f"{token.text:<10} {token.pos_:<10} {token.tag_:<10} {token.lemma_:<10}")

Yetunde    PROPN      NNP        Yetunde   
,          PUNCT      ,          ,         
i          PRON       PRP        I         
'm         AUX        VBP        be        
sorry      ADJ        JJ         sorry     
but        CCONJ      CC         but       
moji       ADJ        JJ         moji      
and        CCONJ      CC         and       
i          PRON       PRP        I         
seem       VERB       VBP        seem      
too        ADV        RB         too       
busy       ADJ        JJ         busy      
to         PART       TO         to        
be         AUX        VB         be        
able       ADJ        JJ         able      
to         PART       TO         to        
go         VERB       VB         go        
shopping   NOUN       NN         shopping  
.          PUNCT      .          .         


In [79]:
@labeling_function(pre = [spacy_])
def more_than_3_adj(sms):
    return SPAM if [token.pos_ for token in nlp(sms.text)].count("ADJ") > 3 else ABSTAIN

In [82]:
lfs = [check, free, call_now, regex_I_am_free, has_many_uppercase_words, has_person, more_than_3_adj]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 4458/4458 [00:25<00:00, 174.95it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.003813,0.001122
free,1,[1],0.043517,0.043517,0.006954
call_now,2,[1],0.004262,0.003813,0.000000
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.006954
has_many_uppercase_words,4,[1],0.184388,0.029385,0.007178
has_person,5,[0],0.050022,0.007851,0.007851
more_than_3_adj,6,[1],0.031853,0.010094,0.001795


## Combining labeling functions into a single model

The goal of labeling functions is not to achieve individually large coverage. Labeling functions are inherently noisy and can make many individual errors. The true utility of labeling functions becomes apparent when multiple functions are combined to form a single model.

We will first build a simple model based on majority voting, and then build a more complex model. 

In [112]:
lfs = [check, free, call_now, regex_I_am_free, has_person, many_rare_words]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

LFAnalysis(L=L_train, lfs=lfs).lf_summary()

100%|██████████| 1114/1114 [00:00<00:00, 25767.40it/s]


,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.010319,0.001570,0.001122
free,1,[1],0.043517,0.043517,0.006954
call_now,2,[1],0.004262,0.001570,0.000000
regex_I_am_free,3,"[0, 1]",0.043517,0.043517,0.006954
has_person,4,[0],0.050022,0.010094,0.010094
many_rare_words,5,[1],0.062135,0.015253,0.008973


In [113]:
LFAnalysis(L=L_test, lfs=lfs).lf_summary()

,j,Polarity,Coverage,Overlaps,Conflicts
check,0,[1],0.012567,0.000898,0.000898
free,1,[1],0.063734,0.063734,0.006284
call_now,2,[1],0.002693,0.000898,0.000000
regex_I_am_free,3,"[0, 1]",0.063734,0.063734,0.006284
has_person,4,[0],0.047576,0.010772,0.010772
many_rare_words,5,[1],0.068223,0.014363,0.008977


In [114]:
from snorkel.labeling.model import MajorityLabelVoter

majority_model = MajorityLabelVoter()
preds_train = majority_model.predict(L=L_train)

In [115]:
preds_train

array([-1,  0,  1, ..., -1, -1, -1])

In [116]:
import numpy as np

labels, counts = np.unique(preds_train, return_counts=True)

for l, c in zip(labels, counts):
    print(f"LABEL: {l}, count: {c}")

LABEL: -1, count: 3847
LABEL: 0, count: 178
LABEL: 1, count: 433


In [117]:
from snorkel.labeling.model import LabelModel

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train=L_train, n_epochs=500, log_freq=100, seed=42)

INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.008]
INFO:root:[100 epochs]: TRAIN:[loss=0.003]
INFO:root:[200 epochs]: TRAIN:[loss=0.002]
INFO:root:[300 epochs]: TRAIN:[loss=0.002]
100%|██████████| 500/500 [00:00<00:00, 3830.92epoch/s]
INFO:root:Finished Training


In [118]:
majority_acc = majority_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Majority voting accuracy:':<25} {majority_acc * 100:.1f}%")

label_model_acc = label_model.score(L=L_test, Y=df_test.label, tie_break_policy="random")["accuracy"]
print(f"{'Probabilistic model accuracy:':<25} {label_model_acc * 100:.1f}%")

Majority voting accuracy: 50.2%
Probabilistic model accuracy: 50.4%


Unfortunately, some data points will not receive any label. It is necessary to filter out these points before sending the labeling result for further processing.

In [119]:
from snorkel.labeling import filter_unlabeled_dataframe
from snorkel.utils import preds_to_probs, probs_to_preds

preds_train, probs_train = label_model.predict(L=L_train, return_probs=True)

df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X=df_train, y=probs_train, L=L_train)
df_train.shape, df_train_filtered.shape

((4458, 2), (676, 2))

As you can see, we were able to quickly prepare labels for about 650 examples (recall that initially no example in the `df_train` set had labels).

The next step will use prepared labels as training data for the actual classifier. We will use simple [logistic regression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), first pre-processing the input data. Since we are working with text, we will use the [word vector representation](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) created based on 5-grams by `CountVectorizer`.

In [120]:
from snorkel.utils import probs_to_preds
from sklearn.feature_extraction.text import CountVectorizer

preds_train_filtered = probs_to_preds(probs=probs_train_filtered)

vectorizer = CountVectorizer(ngram_range=(1, 5))

X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

In [121]:
from sklearn.linear_model import LogisticRegression

sklearn_model = LogisticRegression(C=1e3, solver='lbfgs')
sklearn_model.fit(X=X_train, y=preds_train_filtered)

LogisticRegression(C=1000.0)

In [122]:
print(f"Logistic regression accuracy: {sklearn_model.score(X=X_test, y=df_test.label) * 100:.1f}%")

Logistic regression accuracy: 55.7%


As can be seen, the final model improved the score over the majority vote and the `LabelModel` model.

#### assignment

Complete the above calls with functions that you wrote yourself and check whether your functions improve the quality of the model.

In [125]:
lfs = [hot, contains_money, has_person, more_than_3_adj]

applier = PandasLFApplier(lfs=lfs)
L_train = applier.apply(df=df_train)
L_test = applier.apply(df=df_test)

analysis = LFAnalysis(L=L_train, lfs=lfs)

label_model = LabelModel(cardinality=2, verbose=True)
label_model.fit(L_train, n_epochs=500, log_freq=100, seed = 42)
label_acc = label_model.score(L_test, df_test.label, tie_break_policy="random")["accuracy"]

preds_train, probs_train = label_model.predict(L=L_train, return_probs = True)
df_train_filtered, probs_train_filtered = filter_unlabeled_dataframe(X = df_train, y = probs_train, L = L_train)

preds_train_filtered = probs_to_preds(probs = probs_train_filtered)
vectorizer = CountVectorizer(ngram_range=(1,5))
X_train = vectorizer.fit_transform(df_train_filtered.text.tolist())
X_test = vectorizer.transform(df_test.text.tolist())

model = LogisticRegression(C=1e3, solver = 'lbfgs')
model.fit(X = X_train, y = preds_train_filtered)

accuracy = model.score(X = X_test, y = df_test.label)
print(f"Accuracy: {accuracy * 100:.3f}%") #Model has improved drastically!

100%|██████████| 1114/1114 [00:06<00:00, 170.15it/s]
INFO:root:Computing O...
INFO:root:Estimating \mu...
  0%|          | 0/500 [00:00<?, ?epoch/s]INFO:root:[0 epochs]: TRAIN:[loss=0.003]
INFO:root:[100 epochs]: TRAIN:[loss=0.000]
INFO:root:[200 epochs]: TRAIN:[loss=0.000]
INFO:root:[300 epochs]: TRAIN:[loss=0.000]
100%|██████████| 500/500 [00:00<00:00, 3904.73epoch/s]
INFO:root:Finished Training


Accuracy: 85.009%


## Transforming functions

The idea of a transforming function is to perform an atomic transformation of an instance. For data that is an image, typical transformations include cropping, rotating, and changing the color palette. For text data, you can replace words with synonyms, substitute named entities, cut random pieces of text, etc. In the following example we will find types of named entities occurring in the text, and then prepare a simple transformer that will randomly replace occurrences of the `PERSON` entity

In [126]:
import spacy

nlp = spacy.load('en_core_web_sm')

for doc in nlp.pipe(df_train.text.sample(frac=0.05)):
    print(f"Entities: {[(e.text, e.label_) for e in doc.ents]}")

Entities: [('Yo carlos', 'PERSON'), ('all this weekend', 'DATE')]
Entities: []
Entities: []
Entities: [('2', 'CARDINAL'), ('IT+BOTH', 'FAC')]
Entities: []
Entities: [('Yar', 'PERSON'), ('4', 'CARDINAL'), ('2', 'CARDINAL')]
Entities: []
Entities: []
Entities: []
Entities: [('Solve d', 'ORG'), ('AfterNoon', 'TIME'), ('1,His', 'CARDINAL'), ('2,Police', 'CARDINAL'), ('2', 'CARDINAL'), ('2', 'CARDINAL'), ('2', 'CARDINAL'), ('U r Brilliant', 'ORG')]
Entities: [('WIN', 'ORG'), ('100', 'CARDINAL'), ('every week', 'DATE'), ('NOW Txt', 'WORK_OF_ART'), ('87066', 'CARDINAL')]
Entities: []
Entities: []
Entities: []
Entities: []
Entities: [('1st', 'ORDINAL'), ('TONE', 'ORG'), ('20', 'CARDINAL'), ('every week', 'DATE'), ('just £1.50', 'MONEY'), ('2', 'CARDINAL')]
Entities: [('Keris bin doin', 'PERSON')]
Entities: []
Entities: [('as much as 2 hours', 'CARDINAL')]
Entities: [('House', 'ORG'), ('January', 'DATE')]
Entities: [('today', 'DATE')]
Entities: []
Entities: [('2', 'CARDINAL'), ('first', 'ORDINA

In [130]:
person_entities = []

for doc in nlp.pipe(df_train.text):
    for e in doc.ents:
        if e.label_ == 'PERSON':
            person_entities.append(e.text)
        
person_entities[:10]

['Nah',
 'Melle Melle',
 'Mark',
 'Yummy',
 'Mark',
 'Mallika Sherawat',
 'Matrix3',
 'ShrAcomOrSglSuplt)10',
 'LS1 3AJ',
 'Wah']

In [131]:
from snorkel.augmentation import transformation_function
from snorkel.preprocess.nlp import SpacyPreprocessor

spacy = SpacyPreprocessor(text_field="text", doc_field="doc", memoize=True)

@transformation_function(pre=[spacy_])
def random_person_ner(sms):
    person_ners = [e.text for e in sms.doc.ents]
    
    if person_ners:
        person_to_replace = np.random.choice(person_ners)
        person_to_add = np.random.choice(person_entities)
        sms.text = sms.text.replace(person_to_replace, person_to_add)
    return sms

Another example of transformation could be using WordNet to find synonyms for words. However, this requires downloading a corpus of data

In [132]:
import nltk
from nltk.corpus import wordnet

nltk.download("wordnet")

[nltk_data] Downloading package wordnet to /Users/Kuba/nltk_data...


True

In [133]:
def get_synonym(word):
    
    synsets = wordnet.synsets(word)
    
    if synsets:
        words = [lemma.name() for lemma in synsets[0].lemmas()]
        
        return np.random.choice([w.replace("_", " ") for w in words])


In [157]:
@transformation_function()
def replace_words_with_synonym(sms, num_replacements=5):

    words = sms.text.split()
    
    for _ in range(num_replacements):
        word_idx = np.random.choice(range(len(words)))
        synonym = get_synonym(words[word_idx])
        if synonym:
            words[word_idx] = synonym
        
    sms.text = ' '.join(words)
    return sms

Let us now compare the original text message content with the transformed versions.

In [135]:
# source: https://github.com/snorkel-team/snorkel-tutorials/blob/master/spam/utils.py

from collections import OrderedDict

def preview_tfs(df, tfs):
    transformed_examples = []
    for f in tfs:
        for i, row in df.iterrows():
            transformed_or_none = f(row)
            # If TF returned a transformed example, record it in dict and move to next TF.
            if transformed_or_none is not None:
                transformed_examples.append(
                    OrderedDict(
                        {
                            "TF Name": f.name,
                            "Original Text": row.text,
                            "Transformed Text": transformed_or_none.text,
                        }
                    )
                )
                
    return pd.DataFrame(transformed_examples)


In [136]:
tfs = [random_person_ner, replace_words_with_synonym]

preview_tfs(df_train.sample(frac=0.1), tfs)

,TF Name,Original Text,Transformed Text
0,random_person_ner,Cramps stopped. Going back to sleep,Yar stopped. Going back to sleep
1,random_person_ner,URGENT! We are trying to contact U. Todays draw shows that you have won a £800 prize GUARANTEED. Call 09050001808 from land line. Claim M95. Valid12hrs only,URGENT! We are trying to contact Drop draw shows that you have won a £800 prize GUARANTEED. Call 09050001808 from land line. Claim M95. Valid12hrs only
2,random_person_ner,God's love has no limit. God's grace has no measure. God's power has no boundaries. May u have God's endless blessings always in ur life...!! Gud ni8,God's love has no limit. God's grace has no measure. God's power has no boundaries. Clos1 Lvblefrnd Jstfrnd u have God's endless blessings always in ur life...!! Gud ni8
3,random_person_ner,"Urgent! call 09061749602 from Landline. Your complimentary 4* Tenerife Holiday or £10,000 cash await collection SAE T&Cs BOX 528 HP20 1YF 150ppm 18+",Urgent! call 09061749602 from Landline. Your complimentary 4* Tenerife Holiday or £apps da cash await collection SAE T&Cs BOX 528 HP20 1YF 150ppm 18+
4,random_person_ner,After the drug she will be able to eat.,After the drug she will be able to eat.
...,...,...,...
887,replace_words_with_synonym,Nothing but we jus tot u would ask cos u ba gua... But we went mt faber yest... Yest jus went out already mah so today not going out... Jus call lor...,Nothing but we jus tot u would enquire cos u ba gua... But we went mt faber yest... Yest jus went out already mah soh today not going out... Jus call lor...
888,replace_words_with_synonym,Dai i downloaded but there is only exe file which i can only run that exe after installing.,Dai i downloaded but there be only exe file which i can only run that exe after installing.
889,replace_words_with_synonym,"Say this slowly.? GOD,I LOVE YOU &amp; I NEED YOU,CLEAN MY HEART WITH YOUR BLOOD.Send this to Ten special people &amp; u c miracle tomorrow, do it,pls,pls do it...","Say this slowly.? GOD,I LOVE YOU &amp; I NEED YOU,CLEAN MY HEART WITH YOUR BLOOD.Send this to Ten special people &amp; u c miracle tomorrow, do it,pls,pls do it..."
890,replace_words_with_synonym,I'm home.,I'm home.


Applying transforming functions requires some policy defining the order and number of transformations. In the example below, two transformation functions are drawn at random and this sequence of two functions is applied twice to each data point. As a result, we triple the size of the train set.

In [137]:
from snorkel.augmentation import RandomPolicy, PandasTFApplier

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

100%|██████████| 446/446 [00:02<00:00, 155.81it/s]


In [138]:
df_train_sample.shape, df_train_augmented.shape

((446, 2), (1338, 2))

#### assignment

Modify the transforming function ``replace_words_with_synonym()`` so that you can restrict the replacement of words with synonyms only for specific parts of speech (e.g., replace only nouns or verbs).

In [158]:
@transformation_function()
def replace_words_with_synonym_specific_pos(sms, num_replacements=5):
    words = sms.text.split()
    poss = ["ADJ", "NOUN", "VERB", "PRON"]
    tokens_pos = [token.pos_ for token in nlp(sms.text)]
    it, changes = (0, 0) #we either want 5 changes or 15 tries to make some changes in sms; it is done not to have infinite loops
    
    while it < 3 * num_replacements and changes < num_replacements:
        word_idx = np.random.choice(range(len(words)))
        if (tokens_pos[word_idx] in poss):
            synonym = get_synonym(words[word_idx])
            if synonym:
                changes += 1
                words[word_idx] = synonym
        it += 1
        
    sms.text = ' '.join(words)
    return sms

In [159]:
tfs = [random_person_ner, replace_words_with_synonym_specific_pos]

preview_tfs(df_train.sample(frac=0.1), tfs)

random_policy = RandomPolicy(len(tfs), sequence_length=2, n_per_original=2, keep_original=True)

tf_applier = PandasTFApplier(tfs, random_policy)

df_train_sample = df_train.sample(frac=0.1)
df_train_augmented = tf_applier.apply(df_train_sample)

print(df_train_augmented.text[:15])

100%|██████████| 446/446 [00:07<00:00, 61.70it/s]

2098                                                                                         Are you the cutest girl in the world or what
2098                                                                                    Are you the cut young woman in the cosmos or what
2098                                                                                 Are you the cut young woman in the existence or what
2586                                                                                         I will be outside office take all from there
2586                                                               atomic number John will be outside business office take all from there
2586                                                          atomic number NICHOLS will be outside business office return all from there
5053                      Tick, tick, tick .... Where are you ? I could die of loneliness you know ! *pouts* *stomps feet* I need you ...
5053               Wait.i, tick, t